In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
from tqdm import tqdm

##데이터 다운로드

In [ ]:
# 데이터 압축 풀기
%cd /content/drive/MyDrive/determining_code_similarity_AI_competition/data
!unzip -qq "/content/drive/MyDrive/open.zip"

In [ ]:
# 데이터 개수 확인
import os

def get_files_count(folder_path):
	dirListing = os.listdir(folder_path)
	return len(dirListing)
	
dir_path = "/content/drive/MyDrive/determining_code_similarity_AI_competition/data/code"
cnt = 0
fcnt = {}

for (root, directories, files) in os.walk(dir_path):
  for d in directories:
    f = get_files_count(os.path.join(root, d))
    fcnt[d] = f
    cnt += f

print(fcnt)
print("전체 코드 파일 수: ",cnt)

##데이터 전처리

In [ ]:
import os
import pandas as pd
import re

####sample_train.csv 전처리

In [ ]:
# sample_train.csv 처리
file_path = '/content/drive/MyDrive/determining_code_similarity_AI_competition/data/sample_train.csv'
train = pd.read_csv(file_path, encoding='utf-8')
train.info()

In [ ]:
# train = train.replace(re.compile('(^import.*|^from.*)',re.MULTILINE),"",regex=True) #import,from 없애기
train = train.replace(re.compile('(#.*)', re.MULTILINE),"",regex=True) #주석 한 줄
train = train.replace(re.compile('[\'\"]{3}.*?[\'\"]{3}', re.DOTALL),"",regex=True) #주석 여러줄
train = train.replace(re.compile('[\n]{2,}', re.MULTILINE),"\n",regex=True) #다중개행 한번으로
train = train.replace(re.compile('[ ]{4}', re.MULTILINE),"\t",regex=True) #tab 변환
train = train.replace(re.compile('[ ]{1,3}', re.MULTILINE)," ",regex=True) #공백 여러개 변환
train

##모델 구성

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
import numpy as np
import torch
import datasets
import pickle
import torch,gc
from datasets import load_metric, Dataset, load_dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer

In [ ]:
# train, valid 데이터 분리
from sklearn.model_selection import train_test_split

train_df, valid_df, train_label, valid_label = train_test_split(
        train,
        train['similar'],
        random_state=100,
        test_size=0.1,
        stratify=train['similar'],
        )

train_df.head()

In [ ]:
# DatasetDict 타입으로 변환
train_dataset = Dataset.from_dict(train_df)
valid_dataset = Dataset.from_dict(valid_df)
dataset = datasets.DatasetDict({"train":train_dataset,"valid":valid_dataset})
dataset

In [ ]:
batch_size = 16
epoch_num = 8
MAX_LEN = 256
metric = load_metric("accuracy")
metric_name = "accuracy"

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("microsoft/graphcodebert-base")
tokenizer.truncation_side = 'left'

In [ ]:
def tokenize_function(sample):
    return tokenizer(
        sample['code1'],
        sample['code2'],
        padding=True,
        max_length=MAX_LEN,
        truncation='longest_first',
        return_token_type_ids=True,
        return_attention_mask=True,
        return_length=True,
    )

In [ ]:
# 함수 테스트
tokenize_function(dataset["train"][:5])

In [ ]:
# 토큰화
encoded_dataset = dataset.map(tokenize_function, batched=True)

In [ ]:
with open("/content/drive/MyDrive/determining_code_similarity_AI_competition/data/encoded_dataset", "wb" ) as file:
  pickle.dump(encoded_dataset, file)

In [ ]:
with open("/content/drive/MyDrive/determining_code_similarity_AI_competition/data/encoded_dataset", "rb" ) as file:
  encoded_dataset = pickle.load(file)
  print(encoded_dataset)

In [ ]:
encoded_dataset=encoded_dataset.rename_column(original_column_name='similar',new_column_name='labels')

In [ ]:
# binary classification으로 SequenceClassification 모델 로드
num_labels = 2
model = RobertaForSequenceClassification.from_pretrained("microsoft/graphcodebert-base", num_labels=num_labels)

In [ ]:
args = TrainingArguments(
    "test_GraphCodeBERT",
    evaluation_strategy="epoch", save_strategy="epoch", logging_strategy="epoch", 
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch_num,
    weight_decay=0.01,
    do_train=True,
    do_eval=True,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["valid"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
!pip install knockknock

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from knockknock import discord_sender

@discord_sender(webhook_url="https://discordapp.com/api/webhooks/981434503673286666/SPt7eHcgTwA-1JCfBJNoKkLCigm3LM4ll3DJxSH9Ym4uQNUsr-PalYs38YtSabWz1j4U")
def train_fn():
  trainer.train()
  pass

train_fn()

In [ ]:
trainer.save_model("/content/drive/MyDrive/determining_code_similarity_AI_competition/data/model_1")

In [ ]:
TEST = "/content/drive/MyDrive/determining_code_similarity_AI_competition/data/test.csv"
SUB = "/content/drive/MyDrive/determining_code_similarity_AI_competition/data/sample_submission.csv"

test_dataset = load_dataset("csv", data_files=TEST)['train']
test_dataset = test_dataset.map(tokenize_function, remove_columns=['code1', 'code2'])

predictions = trainer.predict(test_dataset)

df = pd.read_csv(SUB)
df['similar'] = np.argmax(predictions.predictions, axis=-1)
df.to_csv('/content/drive/MyDrive/determining_code_similarity_AI_competition/data/submission.csv', index=False)

In [ ]:
gc.collect()
torch.cuda.empty_cache()